In [2]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle 
import tensorflow


In [3]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
### Preprocess the data 
## Drop the irrelevant data 
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1) #axis=1 -> column , row --> axis =1 
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [12]:
## Encode the catogerical Variables 
label_encode_gender = LabelEncoder ()
data['Gender']=label_encode_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [13]:
from sklearn.preprocessing import OneHotEncoder 
onehot_geo  = OneHotEncoder()
geo_encoder = onehot_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [14]:
geo_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_geo.get_feature_names_out(['Geography']))
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [15]:
data=pd.concat([data.drop('Geography',axis=1),geo_df],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [ ]:
X = data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']


In [17]:
X_train ,X_test , y_train ,y_test = train_test_split (X,y, test_size=0.2, random_state=42)

In [18]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [19]:
## Loa d the encoder and scaler 

with open ('label_encode_gender.pkl','wb') as file:
    pickle.dump(label_encode_gender,file )

with open ('onehot_geo.pkl','wb') as file:
    pickle.dump(onehot_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)


#### ANN Regression Problem statements 

In [20]:
### ANN Regression 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [21]:
# X_train.shape = number of input layer 
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)) , ## First Hidden Layer with input layer 
    Dense(32,activation='relu'),  ## HL 2 with 32 Neurons 
    Dense(1) ## In Output layer we will not set any activation functions as this is regeression problem not the classifications  
    ]

)
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard  

## Set up the tensorboard 
log_dir ="regressionlogs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [24]:
### Set up Early Stopping :
early_stopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

# We are doing early stopping as because we don't want to keep training our model it's trained on lesser epoches .

In [25]:
## Training the models 
history= model.fit(
    X_train,y_train,validation_data = (X_test, y_test) , 
    epochs = 100,
    callbacks=[tensorflow_callback, early_stopping]
)


Epoch 1/100


250/250 [==============================] - 5s 8ms/step - loss: 100368.7812 - mae: 100368.7812 - val_loss: 98484.7656 - val_mae: 98484.7656
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99534.1484 - mae: 99534.1484 - val_loss: 96810.5156 - val_mae: 96810.5156
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 96646.9062 - mae: 96646.9062 - val_loss: 92572.4297 - val_mae: 92572.4297
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 91026.7344 - mae: 91026.7344 - val_loss: 85595.0000 - val_mae: 85595.0000
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 82907.2734 - mae: 82907.2734 - val_loss: 76687.1172 - val_mae: 76687.1172
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 73516.7109 - mae: 73516.7109 - val_loss: 67574.1094 - val_mae: 67574.1094
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 64629.7891 - mae: 64629.789

In [26]:
### Load Tensorboard Extension
%load_ext tensorboard

In [42]:
%tensorboard --logdir logs/fit/20250315-173746

Reusing TensorBoard on port 6008 (pid 10972), started 0:00:10 ago. (Use '!kill 10972' to kill it.)

In [43]:
## Evaluate the models on the test data :
test_loss, test_mae =model.evaluate(X_test,y_test)
print(f'Test MAE :{test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50228.0898 - mae: 50228.0898
Test MAE :50228.08984375


In [44]:
model.save('regression_model.h5')

d:\Udemy\python\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#######Streamlit deployments 